# Prep SLURM commands

In [1]:
getwd()

[1] "/dcs04/lieber/statsgen/mnagle/mwas/CpGWAS/scripts"

In [2]:
setwd("../")

In [3]:
# Constant Arguments Setup
constant_args_df <- data.frame(
  outdir = "./output/",
  chunk1 = 1000000,
  chunk2 = 1001000,
  snp_data_path = "/dcs04/lieber/statsgen/mnagle/mwas/gwas/libd_chr1.pgen",
  methylation_data_path = "/dcs04/lieber/statsgen/mnagle/mwas/pheno/dlpfc/out/chr1_AA.rda",
  verbose = FALSE,
  lambda_choice = "1se",
  alphas = "0,0.25,0.5,0.75,1",
  allow_inefficient_parallelization = FALSE,
  n_fold = 5,
  window_sizes = "1000,2000,5000,10000,20000,50000,100000,500000",
  save_evaluation_results_each_fold = FALSE,
  save_glmnet_object = FALSE,
  cv_eval_mode = "dynamic"
)

# Varying parameters
cv_eval_modes <- c("static", "dynamic")
cores_per_alphas <- c("1", "all") #NA)  # Include NA to signify the default value should be used
num_cores_options <- c(1, 8)

generate_slurm_script <- function(args, tag_pt1, tag_pt2, mem_per_cpu = "16G") {
  cpus_per_task <- if (is.null(args$num_cores) || args$num_cores == "all") {
    "#SBATCH --exclusive\n"
  } else {
    paste0("#SBATCH --cpus-per-task=", args$num_cores, "\n")
  }

  # Always set mem_per_cpu flag, even in exclusive mode
  mem_allocation <- paste0("#SBATCH --mem-per-cpu=", mem_per_cpu, "\n")

  args_string <- paste("--", names(args), "=", args, sep = "", collapse = " ")
  args_string <- paste(args_string, " --tag=", tag_pt1, "-", tag_pt2, sep = "")

slurm_script <- paste(
  "#!/bin/bash\n",
  cpus_per_task,
  mem_allocation,
  "#SBATCH --output=slurm_output_", tag_pt1, "-", tag_pt2, ".out\n",
  "#SBATCH --job-name=", tag_pt1, "-", tag_pt2, "\n",
  "module load conda\n",
  "conda activate mwas\n",
  "echo 'Executing Rscript with arguments: Rscript scripts/CLI.R ", args_string, "'\n",
  "Rscript scripts/CLI.R ", args_string, "\n",
  sep = ""
)

  return(list(script = slurm_script, path = paste0("slurm_scripts/", tag_pt1, "-", tag_pt2, ".sh")))
}

# Overwrite flag (set by user)
overwrite <- FALSE

# Loop through each combination
for (cv_eval_mode in cv_eval_modes) {
  for (cores_per_alpha in cores_per_alphas) {
    for (num_cores in num_cores_options) {
      # Update constant_args_df for the current combination
      constant_args_df$cv_eval_mode <- cv_eval_mode
      constant_args_df$num_cores <- num_cores
      if (!is.na(cores_per_alpha)) {
        constant_args_df$cores_per_alpha <- cores_per_alpha
      } else {
        constant_args_df$cores_per_alpha <- NULL
      }

      # Generate tags
      snp_base <- tools::file_path_sans_ext(basename(constant_args_df$snp_data_path))
      meth_base <- tools::file_path_sans_ext(basename(constant_args_df$methylation_data_path))
      datetime_str <- format(Sys.time(), "%Y%m%d-%H%M%S")
      tag_pt1 <- paste(snp_base, meth_base, format(constant_args_df$chunk1, scientific = FALSE), format(constant_args_df$chunk2, scientific = FALSE), cv_eval_mode, paste0(num_cores, "corestotal"), ifelse(is.na(cores_per_alpha), "defaultcore", paste0(cores_per_alpha, "corepera")), sep = "-")
      tag_pt2 <- datetime_str

      # Check for existing file
      existing_files <- list.files(path = constant_args_df$outdir, pattern = paste0("^", tag_pt1, ".*\\.rds$"), full.names = TRUE)
      if (!overwrite && length(existing_files) > 0) {
        message("File with tag ", tag_pt1, " already exists. Skipping...")
        next
      }

      # Generate and print SLURM script
      script_info <- generate_slurm_script(constant_args_df, tag_pt1, tag_pt2,
                                           mem_per_cpu = "16G")

      cat(script_info$script)
        
      writeLines(script_info$script, script_info$path)

      # Submit the SLURM job using the sbatch command
      system(paste("sbatch", script_info$path))

      # Implement job submission limits and intervals if necessary
      Sys.sleep(1)
      message("ZZZZzzzz....")
    }
  }
}


#!/bin/bash
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=16G
#SBATCH --output=slurm_output_libd_chr1-chr1_AA-1000000-1001000-static-1corestotal-1corepera-20240131-122515.out
#SBATCH --job-name=libd_chr1-chr1_AA-1000000-1001000-static-1corestotal-1corepera-20240131-122515
module load conda
conda activate mwas
echo 'Executing Rscript with arguments: Rscript scripts/CLI.R --outdir=./output/ --chunk1=1e+06 --chunk2=1001000 --snp_data_path=/dcs04/lieber/statsgen/mnagle/mwas/gwas/libd_chr1.pgen --methylation_data_path=/dcs04/lieber/statsgen/mnagle/mwas/pheno/dlpfc/out/chr1_AA.rda --verbose=FALSE --lambda_choice=1se --alphas=0.25,0.5,0.75,1 --allow_inefficient_parallelization=FALSE --n_fold=5 --window_sizes=1000,2000,5000,10000,20000,50000,100000,500000 --save_evaluation_results_each_fold=FALSE --save_glmnet_object=FALSE --cv_eval_mode=static --num_cores=1 --cores_per_alpha=1 --tag=libd_chr1-chr1_AA-1000000-1001000-static-1corestotal-1corepera-20240131-122515'
Rscript scripts/CLI.R --outdi

ZZZZzzzz....



#!/bin/bash
#SBATCH --cpus-per-task=8
#SBATCH --mem-per-cpu=16G
#SBATCH --output=slurm_output_libd_chr1-chr1_AA-1000000-1001000-static-8corestotal-1corepera-20240131-122516.out
#SBATCH --job-name=libd_chr1-chr1_AA-1000000-1001000-static-8corestotal-1corepera-20240131-122516
module load conda
conda activate mwas
echo 'Executing Rscript with arguments: Rscript scripts/CLI.R --outdir=./output/ --chunk1=1e+06 --chunk2=1001000 --snp_data_path=/dcs04/lieber/statsgen/mnagle/mwas/gwas/libd_chr1.pgen --methylation_data_path=/dcs04/lieber/statsgen/mnagle/mwas/pheno/dlpfc/out/chr1_AA.rda --verbose=FALSE --lambda_choice=1se --alphas=0.25,0.5,0.75,1 --allow_inefficient_parallelization=FALSE --n_fold=5 --window_sizes=1000,2000,5000,10000,20000,50000,100000,500000 --save_evaluation_results_each_fold=FALSE --save_glmnet_object=FALSE --cv_eval_mode=static --num_cores=8 --cores_per_alpha=1 --tag=libd_chr1-chr1_AA-1000000-1001000-static-8corestotal-1corepera-20240131-122516'
Rscript scripts/CLI.R --outdi

ZZZZzzzz....



#!/bin/bash
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=16G
#SBATCH --output=slurm_output_libd_chr1-chr1_AA-1000000-1001000-static-1corestotal-allcorepera-20240131-122517.out
#SBATCH --job-name=libd_chr1-chr1_AA-1000000-1001000-static-1corestotal-allcorepera-20240131-122517
module load conda
conda activate mwas
echo 'Executing Rscript with arguments: Rscript scripts/CLI.R --outdir=./output/ --chunk1=1e+06 --chunk2=1001000 --snp_data_path=/dcs04/lieber/statsgen/mnagle/mwas/gwas/libd_chr1.pgen --methylation_data_path=/dcs04/lieber/statsgen/mnagle/mwas/pheno/dlpfc/out/chr1_AA.rda --verbose=FALSE --lambda_choice=1se --alphas=0.25,0.5,0.75,1 --allow_inefficient_parallelization=FALSE --n_fold=5 --window_sizes=1000,2000,5000,10000,20000,50000,100000,500000 --save_evaluation_results_each_fold=FALSE --save_glmnet_object=FALSE --cv_eval_mode=static --num_cores=1 --cores_per_alpha=all --tag=libd_chr1-chr1_AA-1000000-1001000-static-1corestotal-allcorepera-20240131-122517'
Rscript scripts/CLI.R

...

ZZZZzzzz....



#!/bin/bash
#SBATCH --cpus-per-task=1
#SBATCH --mem-per-cpu=16G
#SBATCH --output=slurm_output_libd_chr1-chr1_AA-1000000-1001000-dynamic-1corestotal-allcorepera-20240131-122521.out
#SBATCH --job-name=libd_chr1-chr1_AA-1000000-1001000-dynamic-1corestotal-allcorepera-20240131-122521
module load conda
conda activate mwas
echo 'Executing Rscript with arguments: Rscript scripts/CLI.R --outdir=./output/ --chunk1=1e+06 --chunk2=1001000 --snp_data_path=/dcs04/lieber/statsgen/mnagle/mwas/gwas/libd_chr1.pgen --methylation_data_path=/dcs04/lieber/statsgen/mnagle/mwas/pheno/dlpfc/out/chr1_AA.rda --verbose=FALSE --lambda_choice=1se --alphas=0.25,0.5,0.75,1 --allow_inefficient_parallelization=FALSE --n_fold=5 --window_sizes=1000,2000,5000,10000,20000,50000,100000,500000 --save_evaluation_results_each_fold=FALSE --save_glmnet_object=FALSE --cv_eval_mode=dynamic --num_cores=1 --cores_per_alpha=all --tag=libd_chr1-chr1_AA-1000000-1001000-dynamic-1corestotal-allcorepera-20240131-122521'
Rscript scripts/C

ZZZZzzzz....



#!/bin/bash
#SBATCH --cpus-per-task=8
#SBATCH --mem-per-cpu=16G
#SBATCH --output=slurm_output_libd_chr1-chr1_AA-1000000-1001000-dynamic-8corestotal-allcorepera-20240131-122522.out
#SBATCH --job-name=libd_chr1-chr1_AA-1000000-1001000-dynamic-8corestotal-allcorepera-20240131-122522
module load conda
conda activate mwas
echo 'Executing Rscript with arguments: Rscript scripts/CLI.R --outdir=./output/ --chunk1=1e+06 --chunk2=1001000 --snp_data_path=/dcs04/lieber/statsgen/mnagle/mwas/gwas/libd_chr1.pgen --methylation_data_path=/dcs04/lieber/statsgen/mnagle/mwas/pheno/dlpfc/out/chr1_AA.rda --verbose=FALSE --lambda_choice=1se --alphas=0.25,0.5,0.75,1 --allow_inefficient_parallelization=FALSE --n_fold=5 --window_sizes=1000,2000,5000,10000,20000,50000,100000,500000 --save_evaluation_results_each_fold=FALSE --save_glmnet_object=FALSE --cv_eval_mode=dynamic --num_cores=8 --cores_per_alpha=all --tag=libd_chr1-chr1_AA-1000000-1001000-dynamic-8corestotal-allcorepera-20240131-122522'
Rscript scripts/C

ZZZZzzzz....



## Define parameters

In [4]:
# # Constant Arguments Setup
# constant_args_df <- data.frame(
#   outdir = "./output/",
#   chunk1 = 1000000,
#   chunk2 = 1001000,
#   snp_data_path = "/dcs04/lieber/statsgen/mnagle/mwas/gwas/libd_chr1.pgen",
#   methylation_data_path = "/dcs04/lieber/statsgen/mnagle/mwas/pheno/dlpfc/out/chr1_AA.rda",
#   #snp_data_path = "/Users/michaelnagle/code/mwas/gwas/libd_chr1.pgen",
#   #methylation_data_path = "/Users/michaelnagle/code/mwas/pheno/dlpfc/out/chr1_AA.rda",  
#   verbose = FALSE,
#   lambda_choice = "1se",
#   alphas = "0.25,0.5,0.75,1",
#   #num_cores = "all",
#   allow_inefficient_parallelization = FALSE,
#   n_fold = 5,
#   window_sizes = "1000,2000,5000,10000,20000,50000,100000,500000",
#   #tag = format(Sys.time(), "%Y%m%d-%H%M%S"),
#   save_evaluation_results_each_fold = FALSE,
#   save_glmnet_object = FALSE,
#   cv_eval_mode = "dynamic"
# )

# # Convert alphas to a comma-separated string
# #constant_args_df$alphas <- sapply(constant_args_df$alphas, function(x) paste(x, collapse = ","))


# # Varying parameters
# cv_eval_modes <- c("static", "dynamic")
# cores_per_alphas <- c("all", "1")

## Generate and deploy SLURM calls

In [5]:
# generate_slurm_script <- function(args, tag) {
#   # Check if cores_per_alpha is set and not NULL
#   if (!is.null(args$cores_per_alpha) && args$cores_per_alpha == "all") {
#     mem_allocation <- "0"
#     cpus_per_task <- "#SBATCH --exclusive\n"
#   } else {
#     mem_allocation <- "16G"
#     cpus_per_task <- "#SBATCH --cpus-per-task=1\n"
#   }

#   args_string <- paste("--", names(args), "=", args, sep = "", collapse = " ")
#   args_string <- paste(args_string, " --tag=", tag, sep = "")

#   slurm_script <- paste(
#     "#!/bin/bash\n",
#     cpus_per_task,
#     "#SBATCH --mem=", mem_allocation, "\n",
#     "#SBATCH --output=slurm_output_", tag, ".out\n",
#     "#SBATCH --job-name=", tag, "\n",
#     "Rscript scripts/CLI.R ", args_string, "\n",
#     sep = ""
#   )

#   return(slurm_script)
# }

# # Varying parameters
# cv_eval_modes <- c("static", "dynamic")
# cores_per_alphas <- c("1", NA)  # Include NA to signify the default value should be used

# # Loop through each combination
# for (cv_eval_mode in cv_eval_modes) {
#   for (cores_per_alpha in cores_per_alphas) {
#     # Update constant_args_df for the current combination
#     constant_args_df$cv_eval_mode <- cv_eval_mode
#     # Use an if-statement to decide whether to assign the value or leave it to default
#     if (!is.na(cores_per_alpha)) {
#       constant_args_df$cores_per_alpha <- cores_per_alpha
#     } else {
#       constant_args_df$cores_per_alpha <- NULL  # Setting it to NULL to use the function's default
#     }

#     # Generate tag with consideration for NA
#     snp_base <- tools::file_path_sans_ext(basename(constant_args_df$snp_data_path))
#     meth_base <- tools::file_path_sans_ext(basename(constant_args_df$methylation_data_path))
#     datetime_str <- format(Sys.time(), "%Y%m%d-%H%M%S")
#     cores_tag <- ifelse(is.na(cores_per_alpha), "defaultcore", paste0(cores_per_alpha, "core"))
#     tag <- paste(snp_base, meth_base, cv_eval_mode, cores_tag, datetime_str, sep = "-")

#     # Generate and print SLURM script
#     slurm_script <- generate_slurm_script(constant_args_df, tag)
#     cat(slurm_script, "\n\n")

#     slurm_script_path <- paste0("slurm_scripts/", tag, ".sh")  # Define path to save SLURM script

      
#     # Save SLURM script to a file
#     writeLines(slurm_script, slurm_script_path)

#     # Submit the SLURM job using the sbatch command
#     system(paste("sbatch", slurm_script_path))

#     # Implement job submission limits and intervals if necessary
#     # For example, to wait for 1 second between submissions:
#     Sys.sleep(42)
#     message("ZZZZzzzz....")
#   }
# }
